In [253]:
import pandas as pd
names = pd.read_csv('names.csv')
academics = pd.read_csv('academics.csv')
music = pd.read_csv('music.csv')
clubs = pd.read_csv('clubs.csv')

In [254]:
from py2neo import Graph

In [255]:
from py2neo import Relationship

In [256]:
from py2neo import Node

In [257]:
graph = Graph(":bolt://localhost:7687", auth=("neo4j", "Admin12345"))

In [258]:
personArray = []
courseArray = []
artistArray = []
clubsArray = []

In [259]:
for i in range(len(names)):
    row = names.iloc[i]
    person = Node("Person", student_id = row['student_id'], first = row['first'], last = row['last'], 
                  hometown = row['hometown'], state = row['state'], 
                  age = int(row['age']), fav_artist = row['fav_artist'],
                  fav_class = row['fav_class'], fav_club = row['fav_club'],
                  attends = row['attends'], forge = bool(row['forge_member']))
    graph.create(person)
    personArray.append(person)

In [260]:
for i in range(len(academics)):
    row = academics.iloc[i]
    course = Node("Course", course_id = row['course_id'], full_name = row['full_name'],
                 department = row['department'], professor = row['professor'], dept_building = row['dept_building'], 
                 course_avg_rating = row['course_avg_rating'])
    graph.create(course)
    courseArray.append(course)

In [261]:
for i in range(len(music)):
    row = music.iloc[i]
    artist = Node("Artist", artist_name = row['artist_name'], most_popular_song = row['most_popular_song'],
                 genre = row['genre'], age_of_group = int(row['age_of_group']), platinum_albums = int(row['platinum_albums']))
    graph.create(artist)
    artistArray.append(artist)

In [262]:
for i in range(len(clubs)):
    row = clubs.iloc[i]
    club = Node("Club", club_name = row['club_name'], exec_member = bool(row['exec_member']), 
                member_role = row['member_role'], meeting_place = row['meeting_place'],
                tenure_terms = int(row['tenure_terms']), active_member = bool(row['active_member']))
    graph.create(club)
    clubsArray.append(club)

In [263]:
# UVA node
UVA_node = Node("University", uni_name = 'UVA')
graph.create(UVA_node)

In [264]:
enjoyedClass = Relationship.type("ENJOYED")
listensTo = Relationship.type("LISTENS_TO")
memberOf = Relationship.type("IS_A_MEMBER_OF")
attends = Relationship.type("SCHOOL_ATTENDED")

In [265]:
for person in personArray:
    for course in courseArray:
        if person["fav_class"] == course["course_id"]:
            graph.create(enjoyedClass(person, course))

In [266]:
for person in personArray:
    for artist in artistArray:
        if person["fav_artist"] == artist["artist_name"]:
            graph.create(listensTo(person, artist))

In [267]:
for person in personArray:
    for club in clubsArray:
        if person["fav_club"] == club["club_name"]:
            graph.create(memberOf(person, club))

In [268]:
for artist in artistArray:
    matchArtist = False
    for person in personArray:
        if person["fav_artist"] == artist["artist_name"]:
            matchArtist = True
    if matchArtist == False:
        graph.delete(artist)

In [269]:
for clubs in clubsArray:
    matchClub = False
    for person in personArray:
        if person["fav_club"] == clubs["club_name"]:
            matchClub = True
    if matchClub == False:
        graph.delete(clubs)

In [270]:
for person in personArray:
    if person["attends"] == UVA_node['uni_name']:
        graph.create(attends(person, UVA_node))

In [271]:
for person in personArray:
    if person["forge"]:
        graph.create(memberOf(person, clubsArray[len(clubsArray) - 1]))